In [ ]:
import altair as alt
import pandas as pd

- https://altair-viz.github.io/gallery/line_chart_with_datum.html
- https://altair-viz.github.io/gallery/falkensee.html

In [ ]:
alt.renderers.set_embed_options(renderer="svg")

In [ ]:
DATA: str = "https://raw.githubusercontent.com/dssgPT/Plotting-Good-DSSG/main/desafios/002_Qualidade_do_Ar_em_Portugal_EEA/qualidade_do_ar.csv"

In [ ]:
df = pd.read_csv(DATA, index_col=0)
df["cidade"] = df["cidade"].str.strip()

In [ ]:
df["cidade"].unique()

In [ ]:
cols = [
    "cidade",
    "poluente",
    "unidade_de_nivel_de_poluicao",
    "metrica",
    "nome_estacao",
    "ano",
    "nível_de_poluicao",
]

poluente = "O3"
poluente_domain = [0, 80]
cidade = "Lisboa (greater city)"

df_cities = df.query(
    "poluente == @poluente & metrica == 'Média anual' & cidade == @cidade"
).sort_values(by=["ano", "nome_estacao"])[cols]

In [ ]:
df_cities.head()

In [ ]:
max_value = df_cities["nível_de_poluicao"].max()
max_value

In [ ]:
min_value = df_cities["nível_de_poluicao"].min()
min_value

In [ ]:
df_cities[df_cities["nível_de_poluicao"] == max_value]

In [ ]:
df_cities[df_cities["nível_de_poluicao"] == min_value]

In [ ]:
# df_cities["ano"] = pd.to_datetime(df_cities["ano"], format="%Y")

In [ ]:
FONT: str = "Rubik"

In [ ]:
years = range(df_cities["ano"].min(), df_cities["ano"].max() + 1)

df_bg = pd.DataFrame([{"ano": year} for year in years])

df_bg.head()

In [ ]:
base = alt.Chart(df_cities)

ticks = base.mark_tick().encode(
    x=alt.X(
        "ano:O",
        axis=alt.Axis(
            title=None,
            labelExpr="substring(datum.label, 2, 4)",
            labelAngle=0,
            labelAlign="center",
        ),
    ),
    y=alt.Y(
        "nível_de_poluicao:Q",
        scale=alt.Scale(domain=poluente_domain),
        axis=alt.Axis(title=None),
    ),
)

max_line = base.mark_rule(color="red").encode(y=alt.datum(max_value))
min_line = base.mark_rule(color="red").encode(y=alt.datum(min_value))

bg = (
    alt.Chart(df_bg)
    .mark_rect()
    .encode(
        x=alt.X("ano:O"),
        color=alt.Color(
            "ano:N", legend=None, scale=alt.Scale(range=["lightgray", "white"])
        ),
    )
)

(bg + max_line + min_line + ticks).properties(title=cidade).configure_view(
    strokeWidth=0
).configure_axis(labelFont=FONT)

---